# SampleFlow API examples

In [45]:
import requests
import os
import pandas as pd
import zipfile
import io
import shutil

In [2]:
rest_api_url = "https://circuitseq.iwr.uni-heidelberg.de/api"

## Public API

This can be accessed by anyone without requiring an authorization token.

### `/remaining`

- returns how many samples are still available this week
- doesn't require an authentication token

In [3]:
# requests is the standard Python library for this:
response = requests.get(f"{rest_api_url}/remaining")

In [4]:
# status code is 200 on success, 401 or other if something went wrong:
response.status_code

200

In [5]:
# the request returns data in json format (which is converted to a Python dict by requests):
response.json()

{'message': '', 'remaining': 95}

## Admin API

An authorization token for an admin account is required

In [6]:
# if we try to access an admin endpoint without suitable authentication we get an error:
response = requests.get(f"{rest_api_url}/admin/samples")

In [7]:
response.status_code

401

In [8]:
response.json()

{'msg': 'Missing Authorization Header'}

## Auth token
- you can get create an API token from the admin page
- then export it as an evironment variable on your computer e.g. `export CIRCUITSEQ_API_TOKEN=abca12421...`
- need to add this token to your header when making requests if you want to authenticate yourself
- if your token expires your previously working authorized request will return a 401 or 422 status code
  - generating a new token from the admin page and exporting it should fix this

In [18]:
# get the API token from the environment variable (better not to directly add it to your script)
auth_token = os.environ["CIRCUITSEQ_API_TOKEN"]
# make an authorization header for requests to use
auth_header = {"Authorization": f"Bearer {auth_token}"}

### `/admin/samples`

- returns current samples and previous samples

In [19]:
response = requests.get(f"{rest_api_url}/admin/samples", headers=auth_header)

In [20]:
response.status_code

200

In [21]:
response.json().keys()

dict_keys(['current_samples', 'previous_samples'])

In [24]:
pd.DataFrame(response.json()["current_samples"])

,concentration,date,email,has_reference_seq_zip,has_results_zip,id,name,primary_key,running_option,tube_primary_key
0,456,"Wed, 14 Jun 2023 00:00:00 GMT",RESUBMITTED,True,False,93,test_concentration,23_24_A1,,22_50_A1


### `/admin/zipsamples`

- download a zipfile with a tsv of the samples table and reference sequence fasta files for this week

In [32]:
response = requests.post(f"{rest_api_url}/admin/zipsamples", headers=auth_header)

In [ ]:
response.status_code

In [43]:
# inspect contents of returned zip file:
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
print(f"{zip_file.namelist()}")

['references/', 'samples.tsv', 'references/23_24_A1_test_concentration.zip']


In [44]:
# write downloaded binary data to a file:
with open("samples.zip", "wb") as f:
    f.write(response.content)

### `/admin/result`

- upload success or failure status for results of sample analysis

#### Upload an unsuccessful result

- you need to provide
  - `primary_key`: the primary key of the sample
  - `success="false"`: success status of analysis
- zip file not required
- if the primary key is valid the user will receive an email with the unsuccessful analysis status
- otherwise a diagnostic message will be returned

In [53]:
response = requests.post(
    f"{rest_api_url}/admin/result",
    data={"primary_key": "22_50_A1", "success": "false"},
    headers=auth_header,
)

In [54]:
response.status_code

200

In [55]:
response.json()

{'message': 'Results email for 22_50_A1 sent to liam.keegan@iwr.uni-heidelberg.de'}

#### Upload a successful result

- you need to provide
  - `primary_key`: the primary key of the sample
  - `success="true"`: success status of analysis
  - `file`: a zipfile containing the results (if analysis was successful)
    - if the zipfile contains a text file called `email.txt`
    - any filenames listed in that file will be extracted from the zipfile and emailed to the user
- returns the path on the server where the zipfile is stored
  - the user will also receive an email with fasta/gbk results attached
- on failure a diagnostic message will be returned

In [56]:
# this assumes you have created a zip file named `22_50_A1.zip` containing the results:
with open("22_50_A1.zip", "rb") as f:
    response = requests.post(
        f"{rest_api_url}/admin/result",
        data={"primary_key": "22_50_A1", "success": "true"},
        files={"file": f},
        headers=auth_header,
    )

In [57]:
response.status_code

200

In [58]:
response.json()

{'message': 'Results file saved, Results email for 22_50_A1 sent to liam.keegan@iwr.uni-heidelberg.de'}